In [ ]:
#
import os,io

from multiprocessing import Pipe, Process, set_start_method

class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vai ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe(duplex=True)
        self.timeout=timeout
        
        self.lproc = Process(target=left,  args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)
    
    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start() ; self.rproc.start() 
            self.rproc.join(self.timeout) ; self.lproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()
    

In [ ]:
def X(conn):
    # input stream
    inputs = io.BytesIO(bytes('1'*1024,'utf-8'))

    buffer = bytearray(8)
    while inputs.readinto(buffer):
        conn.send(buffer)
#        _ = conn.recv()

    inputs.close()
    conn.close()




def Y(conn):
     # Inicializa um output stream 
    outputs = io.BytesIO()
    
    while True:
        try:
            buffer = conn.recv()
            outputs.write(buffer)
#            conn.send('ok')
        except EOFError:
            break

    conn.close()
    print(outputs.getvalue())
    outputs.close()



    


In [ ]:
# BiConn(X,Y,timeout=20).manual()

In [ ]:
set_start_method('fork')

In [ ]:
BiConn(Y,X,timeout=100).auto()

In [ ]:
#
import os,io

outputs = io.BytesIO()
outputs.write(bytes('exemplo','utf-8'))
print(outputs.getvalue())
outputs.close()